In [62]:
import mysql.connector
import pandas as pd
import hashlib
import numpy as np
# Connect
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="40418",
    database="aeras"
)
print("✅ Connected!")

# Buffered cursor avoids unread result issues
cursor = conn.cursor(buffered=True)

# Functions
def encoding(input_string):
    return hashlib.sha256(input_string.encode('utf-8')).hexdigest()

def genID(iD, signup, phone, vehicle):
    return signup[2:4] + phone[4:6] + str(iD).zfill(3) + vehicle[-2:]

# Get next ID
cursor.execute("SELECT COUNT(*) FROM points")
iD = cursor.fetchone()[0] + 1


✅ Connected!


In [63]:
df = pd.read_sql("SELECT * FROM points", conn)
print(df)


   id                                           pullerid  point        date  \
0   1  ebc9cb447734823edd404d7f0e279ef099493a0c692f9f...     50  2025-11-13   

     exp_date             time  flaged  
0  2026-05-12  14:27:35.412839       0  


C:\Users\dibya\AppData\Local\Temp\ipykernel_17692\266456092.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM points", conn)


In [64]:
df = pd.read_sql("SELECT * FROM rewards", conn)
print(df)

Empty DataFrame
Columns: [id, pullerid, p_points, redeem, t_points, date]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_17692\1552252940.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM rewards", conn)


In [65]:
pullerid="253400167"
point=50
date="2025-11-13"
from datetime import datetime, timedelta
given_date = datetime.strptime(date, "%Y-%m-%d").date()
# Add 180 days
exp_date = given_date + timedelta(days=180)
time="14:27:35.412839"
flaged=0
redeem=20
trigger=False
baseline=10

In [66]:
pullerid_en=encoding(pullerid)

print(pullerid_en)


ebc9cb447734823edd404d7f0e279ef099493a0c692f9f2cdeb1627b58635662


cursor.execute("ALTER TABLE points CHANGE exp_date date1 VARCHAR(20);")
cursor.execute("ALTER TABLE points CHANGE date exp_date VARCHAR(20);")
cursor.execute("ALTER TABLE points CHANGE date1 date VARCHAR(20);")
conn.commit()

In [67]:
count_query = "SELECT abs_status FROM rides WHERE pullerid = %s"
df = pd.read_sql(count_query, conn, params=[pullerid_en])

total = df["abs_status"].sum()
if(trigger==True and total!=0 and total%5==0):
    point=point+5*baseline

C:\Users\dibya\AppData\Local\Temp\ipykernel_17692\2596721630.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(count_query, conn, params=[pullerid_en])


In [68]:
insert_query = """
INSERT INTO points (pullerid, point,date, exp_date, time, flaged) 
VALUES (%s, %s, %s, %s, %s, %s)
"""
data = (pullerid_en, point,date, exp_date, time, flaged)

cursor.execute(insert_query, data)
conn.commit()

In [69]:
import datetime
import pandas as pd

# 1️⃣ Get today's date in YYYY-MM-DD format
today = datetime.date.today().strftime("%Y-%m-%d")

# 2️⃣ Query all rows where exp_date equals today
query = "SELECT * FROM points WHERE exp_date = %s"
df_exp = pd.read_sql(query, conn, params=[today])
for _, row in df.iterrows():
    exp_date = row['exp_date']
    point_date = row['date']         
    that_points = row['points'] 
 # 4️⃣ Compute sum of all 'redeem' values from point_date to today
    redeem_query = """
        SELECT SUM(redeem) AS total_redeem
        FROM redeem_table
        WHERE date BETWEEN %s AND %s
          AND pullerid = %s
    """
    redeem_df = pd.read_sql(
        redeem_query, conn,
        params=[point_date, exp_date, row['pullerid']]
    )
    total_redeem = redeem_df['total_redeem'].iloc[0] or 0

    # 5️⃣ Compare: if total_redeem < total_points → perform an action
    if total_redeem < that_points:
        point=point-(that_points-total_redeem)


C:\Users\dibya\AppData\Local\Temp\ipykernel_17692\1076407433.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exp = pd.read_sql(query, conn, params=[today])


In [70]:
from datetime import date

today = date.today()

insert_query = """
INSERT INTO rewards (pullerid, p_points, redeem, t_points, date) 
VALUES (%s, %s, %s, %s, %s)
"""

data = (
    pullerid_en,        # puller ID
    point,              # p_points
    redeem,             # redeem
    point - redeem,     # t_points
    today               # date field
)

cursor.execute(insert_query, data)
conn.commit()
print("✅ Reward added")


✅ Reward added


In [71]:
def del_data(db):
    # Always validate the table name (avoid SQL injection)
    allowed_tables = ["points", "pullers", "users","rewards"]
    if db not in allowed_tables:
        raise ValueError("❌ Invalid table name!")

    # Build the SQL statement safely
    del_query = f"TRUNCATE TABLE {db}"
    cursor.execute(del_query)
    conn.commit()

    # Re-fetch the empty table
    df = pd.read_sql(f"SELECT * FROM {db}", conn)
    return df


In [73]:
df_del = del_data("points")
print(df_del)


Empty DataFrame
Columns: [id, pullerid, point, date, exp_date, time, flaged]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_17692\899942194.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {db}", conn)
